In [38]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.integrate import odeint
from scipy.sparse import csc_matrix, csr_matrix, coo_matrix
from qutip import *

In [39]:
# functions to calculate parameters

def A(ra, g, gamma):
    return 2 * ra * g**2 / gamma**2

def B(g, gamma, fA):
    return 4 * fA * g**2 / gamma**2

def BdA(g, gamma):
    return 4 * g**2 / gamma**2

def M(n, m, fBdA):
    return 0.5 * (n + m + 2) + (n - m)**2 * fBdA / 8

def N(n, m, fBdA):
    return 0.5 * (n + m + 2) + (n - m)**2 * fBdA / 16

def n_p(fA, fB, kappa):
    return fA * (fA - kappa) / kappa / fB

In [40]:
# parameters

w_c = 2.0 * np.pi
w_a = 2.0 * np.pi
g = 0.1 * 2* np.pi

gamma = 0.01
kappa = 0.01
ra = 0.01

N_max = 5

fA = A(ra, g, gamma)
fB = B(g, gamma, fA)
fBdA = BdA(g, gamma)
aver_n = n_p(fA, fB, kappa)

In [41]:
fA, fB, kappa

(78.95683520871485, 1246836.3652352307, 0.01)

In [42]:
fB/fA, fBdA

(15791.367041742971, 15791.367041742971)

In [43]:
# average photon number for steady state
aver_n

0.4999366742602235

In [44]:
# coefficients of the differential equation

def fnm(n, m, fA, fBdA, kappa):
    return - (M(n, m, fBdA) * fA / (1 + N(n, m, fBdA) * fBdA)) - 0.5 * kappa * (n + m)

def gnm(n, m, fA, fBdA):
    return np.sqrt(n * m) * fA / (1 + N (n - 1, m - 1, fBdA) * fBdA)

def hnm(n, m, kappa):
    return kappa * np.sqrt((n + 1) * (m + 1))

In [45]:
# parameters in the differential equation
ij = range(N_max)
FNM = np.array([fnm(i, j, fA, fBdA, kappa) for i in ij for j in ij]).reshape(N_max, N_max)
GNM = np.array([gnm(i, j, fA, fBdA) for i in ij for j in ij]).reshape(N_max, N_max)
HNM = np.array([hnm(i, j, kappa) for i in ij for j in ij]).reshape(N_max, N_max)

In [46]:
FNM

array([[-0.00499968, -0.01499241, -0.01999747, -0.02499859, -0.02999905],
       [-0.01499241, -0.01499984, -0.02498737, -0.0299962 , -0.03499803],
       [-0.01999747, -0.02498737, -0.02499989, -0.03498233, -0.03999494],
       [-0.02499859, -0.0299962 , -0.03498233, -0.03499992, -0.04497731],
       [-0.02999905, -0.03499803, -0.03999494, -0.04497731, -0.04499994]])

In [47]:
FNM.reshape(-1)

array([-0.00499968, -0.01499241, -0.01999747, -0.02499859, -0.02999905,
       -0.01499241, -0.01499984, -0.02498737, -0.0299962 , -0.03499803,
       -0.01999747, -0.02498737, -0.02499989, -0.03498233, -0.03999494,
       -0.02499859, -0.0299962 , -0.03498233, -0.03499992, -0.04497731,
       -0.02999905, -0.03499803, -0.03999494, -0.04497731, -0.04499994])

In [84]:
# differential equation
def rho_nm_dot(rho_nm, t):
    rho = rho_nm.reshape(N_max, N_max)
    rho_new = np.zeros([N_max, N_max])
    
    ij = range(N_max)
    for i in ij:
        for j in ij:
            rho_new[i, j] += FNM[i, j] * rho[i, j]
            if i > 0 and j > 0:
                rho_new[i, j] += GNM[i, j] * rho[i - 1, j - 1]
            if i < N_max - 1 and j < N_max - 1:
                rho_new[i, j] += HNM[i, j] * rho[i + 1, j + 1]
    
    return rho_new.reshape(-1)

In [85]:
t_list = np.linspace(0, 20, 101)
rho_0 = np.zeros(N_max * N_max)
rho_0[0] = 1

In [86]:
t_list

array([  0. ,   0.2,   0.4,   0.6,   0.8,   1. ,   1.2,   1.4,   1.6,
         1.8,   2. ,   2.2,   2.4,   2.6,   2.8,   3. ,   3.2,   3.4,
         3.6,   3.8,   4. ,   4.2,   4.4,   4.6,   4.8,   5. ,   5.2,
         5.4,   5.6,   5.8,   6. ,   6.2,   6.4,   6.6,   6.8,   7. ,
         7.2,   7.4,   7.6,   7.8,   8. ,   8.2,   8.4,   8.6,   8.8,
         9. ,   9.2,   9.4,   9.6,   9.8,  10. ,  10.2,  10.4,  10.6,
        10.8,  11. ,  11.2,  11.4,  11.6,  11.8,  12. ,  12.2,  12.4,
        12.6,  12.8,  13. ,  13.2,  13.4,  13.6,  13.8,  14. ,  14.2,
        14.4,  14.6,  14.8,  15. ,  15.2,  15.4,  15.6,  15.8,  16. ,
        16.2,  16.4,  16.6,  16.8,  17. ,  17.2,  17.4,  17.6,  17.8,
        18. ,  18.2,  18.4,  18.6,  18.8,  19. ,  19.2,  19.4,  19.6,
        19.8,  20. ])

In [87]:
rho_0

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [88]:
result = odeint(rho_nm_dot, rho_0, t_list)

In [89]:
len(result)

101

In [95]:
result[100].reshape(N_max, N_max)

array([[  9.13356598e-01,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   8.27763914e-02,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   3.75106804e-03,
          0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.13326763e-04,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   2.56019197e-06]])

In [101]:
# ordinary differential equation for the diagonal terms

def pn_dot(pn, t):
    pn_new = np.zeros(N_max)
    for n in xrange(N_max):
        pn_new[n] += (- (n + 1) * fA / (1 + (n + 1) * fBdA)  - kappa * n) * pn[n]
        if n > 0:
            pn_new[n] += n * fA / (1 + n * fBdA) * pn[n-1]
        if n < N_max - 1:
            pn_new[n] += kappa * (n+1) * pn[n + 1]
    return pn_new

In [102]:
t_list = np.linspace(0, 20, 101)
pn_0 = np.zeros(N_max)
pn_0[0] = 1

In [103]:
result2 = odeint(pn_dot, pn_0, t_list)

In [104]:
result2[100]

array([  9.13356598e-01,   8.27763914e-02,   3.75106804e-03,
         1.13326763e-04,   2.56019197e-06])

In [107]:
coherent_dm(5, 0.5)

Quantum object: dims = [[5], [5]], shape = [5, 5], type = oper, isherm = True
Qobj data =
[[  7.78800837e-01   3.89399874e-01   1.37680507e-01   3.96822144e-02
    1.03552599e-02]
 [  3.89399874e-01   1.94699665e-01   6.88401575e-02   1.98410795e-02
    5.17762272e-03]
 [  1.37680507e-01   6.88401575e-02   2.43398841e-02   7.01523054e-03
    1.83065731e-03]
 [  3.96822144e-02   1.98410795e-02   7.01523054e-03   2.02192662e-03
    5.27631232e-04]
 [  1.03552599e-02   5.17762272e-03   1.83065731e-03   5.27631232e-04
    1.37687844e-04]]